<a href="https://colab.research.google.com/github/DallasAutumn/datawhale_salons/blob/master/NLP_s8_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-s8-task3 特征选择

## 1.TF-IDF原理

首先tf（term frequency）很好理解，就是词语t在文档d中出现的频率。

而idf（inverse document frequency）逆向文档频率的计算公式是$IDF(t)=log\frac{文章总数}{包含单词t的文章数+1}$，其中+1作平滑作用，防止出现对数值取0。直观的解释是，如果一个单词在非常多的文章里都出现，那么它可能是一个比较通用的词汇，对于区分某篇文章特殊语义的贡献较小，因此idf得分就会较低。

综合考虑这两个指标，就得到了tf-idf权重。
$$tf-idf = tf * idf$$

## 2.基于词袋模型的文本矩阵化

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

import jieba
import re

documents = [
    "上面提到，N-gram的NNN越大，模型 Perplexity 越小，表示模型效果越好。这在直观意义上是说得通的，毕竟依赖的词越多，我们获得的信息量越多，对未来的预测就越准确。然而，语言是有极强的创造性的（Creative），当NNN变大时，更容易出现这样的状况：某些n-gram从未出现过，这就是稀疏问题。",
    "n-gram最大的问题就是稀疏问题（Sparsity）。例如，在bi-gram中，若词库中有20k个词，那么两两组合（C220kC_{20k}^2C20k2​）就有近2亿个组合。其中的很多组合在语料库中都没有出现，根据极大似然估计得到的组合概率将会是0，从而整个句子的概率就会为0。最后的结果是，我们的模型只能计算零星的几个句子的概率，而大部分的句子算得的概率是0，这显然是不合理的。",
]

def docs2corpus(docs):
    texts = [jieba.lcut(re.sub("[^\u4e00-\u9fa5]", '', doc)) for doc in docs]
    corpus = [' '.join(text) for text in texts]
    return corpus

corpus = docs2corpus(documents)
print(corpus)

['上面 提到 的 越 大 模型 越小 表示 模型 效果 越好 这 在 直观 意义 上 是 说 得 通 的 毕竟 依赖 的 词越 多 我们 获得 的 信息量 越多 对 未来 的 预测 就 越 准确 然而 语言 是 有 极强 的 创造性 的 当 变大时 更 容易 出现 这样 的 状况 某些 从未 出现 过 这 就是 稀疏 问题', '最大 的 问题 就是 稀疏 问题 例如 在 中 若 词库 中有 个 词 那么 两 两 组合 就 有 近 亿个 组合 其中 的 很多 组合 在 语料库 中 都 没有 出现 根据 极大 似然 估计 得到 的 组合概率 将会 是 从而 整个 句子 的 概率 就 会 为 最后 的 结果 是 我们 的 模型 只能 计算 零星 的 几个 句子 的 概率 而 大部分 的 句子 算得 的 概率 是 这 显然 是 不合理 的']


In [11]:
tfidf_vect = TfidfVectorizer()
X_tfidf = tfidf_vect.fit_transform(corpus)
features = tfidf_vect.get_feature_names()
feat_mat = X_tfidf.toarray()
print(features)
print(feat_mat)

['上面', '不合理', '中有', '亿个', '从未', '从而', '估计', '似然', '例如', '依赖', '信息量', '其中', '准确', '几个', '出现', '创造性', '变大时', '句子', '只能', '大部分', '容易', '将会', '就是', '很多', '得到', '意义', '我们', '提到', '效果', '整个', '显然', '最后', '最大', '未来', '极大', '极强', '某些', '根据', '概率', '模型', '毕竟', '没有', '然而', '状况', '直观', '稀疏', '算得', '组合', '组合概率', '结果', '获得', '表示', '计算', '词库', '词越', '语料库', '语言', '越多', '越好', '越小', '这样', '那么', '问题', '零星', '预测']
[[0.17388037 0.         0.         0.         0.17388037 0.
  0.         0.         0.         0.17388037 0.17388037 0.
  0.17388037 0.         0.24743463 0.17388037 0.17388037 0.
  0.         0.         0.17388037 0.         0.12371731 0.
  0.         0.17388037 0.12371731 0.17388037 0.17388037 0.
  0.         0.         0.         0.17388037 0.         0.17388037
  0.17388037 0.         0.         0.24743463 0.17388037 0.
  0.17388037 0.17388037 0.17388037 0.12371731 0.         0.
  0.         0.         0.17388037 0.17388037 0.         0.
  0.17388037 0.         0.17388037 0.17388037 0.17388

## 3.互信息的原理

$$I(X,Y) = \sum_{x \in X}\sum_{y \in Y} p(x,y) log\frac{p(x,y)}{p(x)p(y)}$$

## 4.利用互信息进行特征筛选

了解了互信息的原理和计算之后，接下来只要使用sklearn.feature_selection模块中的SelectKBest和mutual_info_classif即可解决。